In [1]:
import pandas as pd
dataset_path = "./crime_rankings_2020.csv"

In [2]:
df = pd.read_csv(dataset_path)
df.head()

,rank,country,safety_index
0,1,Venezuela,15.5
1,2,Papua New Guinea,18.1
2,3,South Africa,22.5
3,4,Afghanistan,23.8
4,5,Honduras,23.9
